### Closures

Recall from the lecture that a **closure** is basically a function together with an environment that contains some values (the captured, or **free** variables).

Let's start with a very simple example.

In [1]:
def outer(a, b):
    sum_ = a + b
    def inner():
        prod = a * b
        print(a, b, sum_, prod)
        return "You just called a closure!"
    return inner

As you can see calling `outer` will return a **function** that also "captures" `a`, `b` and `sum_` where `a` and `b` were arguments passed to the `outer` function and `sum_`was a variable created in the `outer` function. Since `a`, `b` and `sum_` were not created in `inner` but are referenced, they need to come from somewhere - and that somewhere is the outer scope.

In [2]:
func = outer(2, 3)

In [3]:
func

<function __main__.outer.<locals>.inner()>

As you can see `func` is actually the returned function `inner` - but it is a closure with free variables `a`, `b` and `sum_` that are `2`, `3` and `5` respectively.

So, we can now **call** `func`:

In [4]:
prod = func()

2 3 5 6


In [5]:
prod

'You just called a closure!'

So we are basically using `outer` like a function **factory** which returns not just a function, but a closure, in this case.

In fact, we can look at the `__closure__` property of that closure:

In [6]:
func.__closure__

(<cell at 0x000001AEA8AA4130: int object at 0x00007FF9563249D8>,
 <cell at 0x000001AEA84F7130: int object at 0x00007FF9563249F8>,
 <cell at 0x000001AEA893C7C0: int object at 0x00007FF956324A38>)

Those **cells** are actually the captured variables - three integers in this case.

In the following case we do **not** have a closure returned:

In [7]:
def outer(a, b):
    def inner(c):
        return c ** 2
    return inner

In [8]:
func = outer(1, 2)

In [9]:
func.__closure__

As you can see there were no "captured" (free) variables here.

So `func` is still the `inner` function, but that is not a closure (there are no variables that were "captured" by `inner`).

Closures serve a critical role in creating Python decorators that we will study in an upcoming chapter.

We saw a simple application of closures in the lecture:

In [10]:
def power(n):
    def inner(x):
        return x ** n
    return inner

As you can see the `inner` function that is returned by calling `power` is going to be a closure, since `n` is a free variable in `inner`:

In [11]:
square = power(2)
cube = power(3)

So `square` is a closure that will return `x**n` where `n` is fixed to `2`, and `cube` will be a closure that also returns `x ** n` but with `n` fixed to `3`:

In [12]:
square(4)

16

In [13]:
cube(3)

27

Free variables in a closure can be any object, including a function.

Let's see a completely useless example that illustrates this as simply as possible first:

In [14]:
def execute(func):
    def inner(a, b):
        result = func(a, b)
        return result
    return inner

So here, `inner` contains the free variable `func` , which we are going to use to pass a function. `inner` will be returned by `execute`, and we can call `func` by calling `inner` with some arguments:

In [15]:
def add(a, b):
    print('running add...')
    return a + b

In [16]:
add_executor = execute(add)

So `inner`'s free variable `func` is the `add` function.

In [17]:
add_executor(2, 3)

running add...


5

Now, `inner` is restrictive as to what arguments can be passed to it: two mandatory positional arguments, which means that `func` must also be a function that takes two positional arguments.

We can this a lot more generic, by using `*args` and `**kwargs`:

In [18]:
def execute(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return inner

In [19]:
def add(a, b, c):
    print('add...')
    return a + b + c

def say_hello(name, *, formal=True):
    print('say_hello...')
    if formal:
        return f'Pleased to meet you, {name}'
    else:
        return f'Hi, {name}!'

In [20]:
exec_add = execute(add)
exec_greet = execute(say_hello)

In [21]:
exec_add(1, 2, 3)

add...


6

In [22]:
exec_greet('Michael', formal=False)

say_hello...


'Hi, Michael!'

So by using `*args` and `**kwargs` we can basically handle passing arguments to any `func` - of course we have to pass the appropriate parameters for whichever `func` is in the closure.

So why is this useful?

We'll come back to this with decorators, but consider a situation where we want to time how long certain functions take to run.

We might have these two functions:

In [23]:
def factorial(n):
    prod = 1
    for i in range(2, n+1):
        prod = prod * i
    return prod

In [24]:
def diagonal_matrix(rows, cols, *, diagonal=1):
     return [
         [
             diagonal if row == col else 0 
             for col in range(cols)
         ] 
         for row in range(rows)
     ]        

In [25]:
factorial(4)

24

In [26]:
diagonal_matrix(3, 3, diagonal=-1)

[[-1, 0, 0], [0, -1, 0], [0, 0, -1]]

So if we now want to time these two functions, we might write code like this:

In [27]:
from time import perf_counter

In [28]:
start = perf_counter()
result = factorial(10_000)
end = perf_counter()
print(f'elapsed: {end - start}')
print(f'result = {result}')

elapsed: 0.03707360000407789


ValueError: Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit

And if we want to time `diagonal_matrix` we wold have to repeat essentially the same code, except which function (and arguments) to use:

In [29]:
start = perf_counter()
result = diagonal_matrix(10, 10, diagonal=-1)
end = perf_counter()
print(f'elapsed: {end - start}')
print(f'result = {result}')

elapsed: 0.00010080001084133983
result = [[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, -1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, -1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, -1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, -1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, -1]]


We really don't want to be writing timing code like this every time we need to time a function - we may want to do this hundreds of times in our program.

Our goal is to use another function to wrap the timing code around our own function, execute the function, returning the result, and printing the timing information out.

We could start with something like this:

In [30]:
def time_it(func, *args, **kwargs):
    start = perf_counter()
    result = func(*args, **kwargs)
    end = perf_counter()
    print(f'elapsed: {end - start}')
    return result

In [31]:
result = time_it(factorial, 10_000)

elapsed: 0.03692189999856055


In [32]:
result

ValueError: Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit

And we can use the exact same function to time the diagonal matrix generator:

In [33]:
result = time_it(diagonal_matrix, 10, 10, diagonal=-1)

elapsed: 1.5599987818859518e-05


In [34]:
result

[[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, -1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, -1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, -1]]

But one thing that is not very nice about this, is that we have to remember to call `time_it(func)` every time we want to use `func`.

So, instead, let's use a closure to make this whole process a little more natural, and easier to read:

In [35]:
def time_it(func):
    def inner(*args, **kwargs):
        # inner is actually going to time and run func
        start = perf_counter()
        result = func(*args, **kwargs)  # func is a free variable
        end = perf_counter()
        print(f'elapsed: {end - start}')
        return result
    return inner

So now, we can create new functions that are the timed versions of our original functions:

In [36]:
timed_fact = time_it(factorial)
timed_diagonal = time_it(diagonal_matrix)

In [37]:
result = timed_fact(5)

elapsed: 2.900007530115545e-06


In [38]:
result = timed_fact(100_000)

elapsed: 4.49383229999512


In [39]:
result = timed_diagonal(10, 10, diagonal=-1)

elapsed: 2.3499989765696228e-05


We'll actually come back to this very example (and others) when we study decorators. But the above application of closures is a very common one in Python.